# Quantitative Finance: Multi-Method Derivative Pricing & Stochastic Modeling
**Overview:** This project implements a comprehensive framework for financial asset simulation and option pricing. 
It covers historical data analysis, multivariate stochastic path generation, and valuation using Black-Scholes, Monte Carlo, and Binomial Tree methods.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.special import comb

## Phase 1: Historical Data Analytics & Parameter Estimation
Extracting log returns, mean vectors, and covariance matrices from historical stock prices to calibrate stochastic models.

In [ ]:
data = pd.read_csv('stock-data.csv', header = 0, index_col = 0)
prices = data.values
log_prices = np.log(prices)
dZ = np.diff(log_prices, axis=0)
final = prices[-1,:]
m = np.mean(dZ, axis = 0)
Sigma = np.cov(dZ, rowvar = False) #np.cov：row:Variable/Asset，column:Observation/Time
Sigma

## Phase 2: Multivariate Stochastic Path Simulation
Implementing Geometric Brownian Motion (GBM) using Cholesky Decomposition to simulate correlated asset paths.

In [ ]:
def simulate_S(S0, m, Sigma, n_steps, n_scenarios, num_years=1):
    assets = S0.shape[0]       # because of the Question5(Monte Carlo Pricing), we couldn't fix the number of assets in 2 if we want to reuse this function.
    annual_mean = m * n_steps
    annual_cov = Sigma * n_steps
    L = np.linalg.cholesky(annual_cov)
    
    final = np.zeros((n_scenarios, num_years + 1, assets))
    final[:, 0, :] = S0
    
    Zt = np.log(S0)[:,None]
    for t in range(num_years):
        epsilon = np.random.randn(assets, num_years, n_scenarios)
        epsilon_t = epsilon[:, t, :] # t year's epsilon
        correlated_noise = L @ epsilon_t # matrix multiplication, '*' is element multiple
        delta_Z = annual_mean[:, None] + correlated_noise # (2,) to (2,1)
        Zt = Zt + delta_Z
        St = np.exp(Zt)
        final[:, t+1:, :] = St.T[:,None,:] # St is (2,1); final is (1,1,2)
    return final

n_scenarios = 10**6
num_years = 1
n_steps = 52
simulated_S = simulate_S(final, m, Sigma, n_steps, n_scenarios, num_years)
simulated_S

In [ ]:
num_years = 5
n_scenarios = 10**6
n_steps = 52

simulated_S_T = simulate_S(final, m, Sigma, n_steps, n_scenarios, num_years)

big_bank = simulated_S_T[:, :, 1]

alpha = np.percentile(big_bank, 9, 0)
beta = np.percentile(big_bank, 50, 0)
gamma = np.percentile(big_bank, 91,0)

times = np.arange(num_years + 1)
plt.plot(times, alpha, label='9th Percentile')
plt.plot(times,beta, label='Median')
plt.plot(times, gamma, label='91th Percentile')

plt.plot(times,simulated_S_T[0,:, 1], label='Sample Path');
plt.xlim(0,6)
plt.title('Fan Diagram for Big Bank')
plt.xlabel('Years')
plt.ylabel('Stock Price')

plt.legend()
plt.show()

## Phase 3: Comparative Option Valuation
1. Analytical Method (Black-Scholes) 
2. Numerical Method (Monte Carlo) 
3. Lattice Method (Binomial Tree)

In [ ]:
def N(x):
    return scipy.stats.norm.cdf(x)

def d1_d2(S0, K, T, r, sigma):

    d1 = 1/(sigma*sqrt(T))*(log(S0/K) + (r+0.5*sigma**2)*T)
    d2 = d1 - sigma*sqrt(T)
    return d1,d2

def black_scholes_call_price(S0,K,T,r,sigma):
    d1, d2 = d1_d2(S0,K,T,r,sigma)
    return S0*N(d1) - exp(-r*(T))*K*N(d2)

In [ ]:
def monte_carlo_pricer(n_scenarios, S0, sigma, r, T, K, option_type):
    Z = np.random.randn(n_scenarios)
    drift = (r - 0.5 * sigma**2) * T
    diffusion = sigma * np.sqrt(T) * Z
    St = S0 * np.exp(drift + diffusion)
    if option_type == 'call':
        payoff = np.maximum(St-K, 0)
    elif option_type == 'put':
        payoff = np.maximum(K-St, 0)
    option_price = exp(-r*T)*np.mean(payoff)
    return option_price

n_scenarios = 10**6
K = 83.0
T = 4
S0 = 121.0
sigma = 0.13
r = 0.01
option_type = 'call'
mc_call_price = monte_carlo_pricer(n_scenarios, S0, sigma, r, T, K, option_type)

n_scenarios = 10**6
K = 83.0
T = 4
S0 = 121.0
sigma = 0.13
r = 0.01
option_type = 'put'
mc_put_price = monte_carlo_pricer(n_scenarios, S0, sigma, r, T, K, option_type)

In [ ]:
def binomial_pricer(K, T, S0, r, u, d, n_steps, option_type='call'):
    dt = T / n_steps
    R = 1 + r * dt # according to E-mail
    discount = 1.0 / R
    p = (R - d) / (u - d)
    
    values = []
    
    for i in range(n_steps + 1):
        St = S0 * (u ** i) * (d ** (n_steps - i))
        if option_type == 'call':
            payoff = max(St - K, 0.0)
        elif option_type == 'put':
            payoff = max(K - St, 0.0)
        else:
            return None
        values.append(payoff)
 
    for step in range(n_steps - 1, -1, -1):
        past_values = []
        for i in range(step + 1):
            value_up = values[i+1]
            value_down = values[i]
            expected_values = p * value_up + (1 - p) * value_down
            present_values = discount * expected_values
            past_values.append(present_values)

        values = past_values

    return values[0]

## Phase 4: Arbitrage Analysis
Arbitrage-Free Check: Validating the Put-Call Parity to ensure market consistency.

In [ ]:
def put_call_parity(C, P, S0, K, r, T):
    left = C + K * np.exp(-r*T)
    right = P + S0
    parity = left - right
    assert abs(parity) < 0.1
    print(f'put call parity holds')
    return

In [ ]:
K = 83.0
T = 4
S0 = 121.0
sigma = 0.13
r = 0.01

#1 BS

C = black_scholes_call_price(S0,K,T,r,sigma)
P = black_scholes_put_price(S0,K,T,r,sigma)

parity_BS = put_call_parity(C, P, S0, K, r, T)

#2 Monte carlo
n_scenarios = 1000000
option_type = 'call'
C = monte_carlo_pricer(n_scenarios, S0, sigma, r, T, K, option_type)

n_scenarios = 1000000
option_type = 'put'
P = monte_carlo_pricer(n_scenarios, S0, sigma, r, T, K, option_type)
parity_Mc = put_call_parity(C, P, S0, K, r, T)

#3 Binomial tree

n_steps = 1000
dt = T / n_steps
u = np.exp(sigma*np.sqrt(dt))
d = np.exp(-sigma*np.sqrt(dt))

C = binomial_pricer(K, T, S0, r, u, d, n_steps, option_type='call')
P = binomial_pricer(K, T, S0, r, u, d, n_steps, option_type='put')
parity_Bt = put_call_parity(C, P, S0, K, r, T)